<a href="https://colab.research.google.com/github/houssemabdelwahed/3-levels-Data-classification-/blob/main/ThreeLevels_SimilarityClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Similarity Classification - EA CP - Complete Model modified for sectors**
### *Pre-trained model for Topic, Subtopic and Sector*



---

> To run each cell you have to click on the button *play* in the left side.

> Atention: Users **don't need to change anything** in the code!



> The code asks you for permissions to enter Google Drive



---

In [ ]:
#@title **1. Download the general model**
%%capture
pip install -U sentence-transformers

In [ ]:
#@title **2. Libraries installation**
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')
embeddings = model.encode(sentences)
print(embeddings)
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

[[ 0.11453557  0.07683481  0.02626466 ... -0.132319   -0.00558201
   0.3162339 ]
 [ 0.00646671  0.1654482  -0.03636224 ...  0.18916464  0.20142542
   0.24428217]]


In [ ]:
Training data base  = "/content/Training model Data Base for Cp  - Sheet1 (2).csv" #@param {type:"string"}
#@markdown ---
# Load dataset to Pandas
df = pd.read_csv(base_clasificada)
max_r = 400000
df = df.iloc[-max_r:]
df = df.rename(columns={'IDEA':'idea'})
df = df.dropna(subset=['idea'])
df.head()

,idea,Topic,Subtopic,Sector
0,Consider single mothers for houses constructio...,Economic support,Single mothers,Protection
1,Support child mothers with money to help us bu...,Economic support,Single mothers,Protection
2,Support single mothers with plastic sheetings ...,Economic support,Single mothers,Protection
3,Support pregnant school girls with money to bu...,Economic support,Single mothers,Protection
4,Stop giving gifts to women at women centre aft...,Economic support,Single mothers,Protection


In [ ]:
#@title **4.1. Model Training**
## Calculate dinstance between all pairs of sentences in DF
e_col = 'sentence_embedding_bert'

## Predictions
df['idea'] = df['idea'].apply(lambda x: x.split('#')[0])
predictions = df
predictions = predictions[['idea']]
predictions['sentence_embedding_bert'] = predictions['idea'].apply(lambda x: model.encode(x))

CPU times: user 28.6 s, sys: 41.9 ms, total: 28.6 s
Wall time: 32.1 s


<timed exec>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#@title **4.2. Model Training**
def get_sim_df_for_iloc(sent_iloc, predictions=predictions,e_col=e_col, pipe=model):
  embed_mat = np.array([x for x in predictions[e_col]])
  sim_mat = cosine_similarity(embed_mat,embed_mat)
  df['sim_score'] = sim_mat[sent_iloc]
  return df
def get_sim_df_total( predictions,e_col, string_to_embed,pipe=model):
  embed_mat = np.array([x for x in predictions[e_col]])
  sim_mat = cosine_similarity(embed_mat,embed_mat)
  for i,v in enumerate(sim_mat):
    s = predictions.iloc[i].idea
    predictions[s] = sim_mat[i]
  return predictions
def get_sim_df_for_string(predictions,e_col, string_to_embed,pipe=model):
  embed_mat = np.array([x for x in predictions[e_col]])
  embedding = pipe.encode(string_to_embed)
  m = np.array([embedding,]*len(df))
  sim_mat = cosine_similarity(m,embed_mat)
  df['sim_score'] = sim_mat[0]
  return df

In [ ]:
#@title **4.3. Model Training**
sentence_to_compare=0
sim_df_for_one_sent = get_sim_df_for_iloc(sentence_to_compare,predictions,e_col)
sim_df_for_one_sent.sort_values('sim_score', ascending = False)
def viz_sim_df_for_one_sent( question='Default', e_col='sentence_embedding_bert', N = 20, sim_df_for_one_sent=sim_df_for_one_sent):
  # Plots the N most similar sentences in our dataframe for sentence at position sent_iloc
  sim_df = get_sim_df_for_string(predictions,e_col,question )
  sim_df.index = sim_df.idea
  ax = sim_df.sort_values('sim_score', ascending = False).iloc[:N][['sim_score','idea']].plot.barh(title = f"Most similar Sentences for sentence\n'{question}'", figsize=(6,6))
  ax.set_xlim(0.1, 1)

In [ ]:
#@title **5. Loads databases to be classified**
Data = "/content/23 - results-20230524-110847.csv" #@param {type:"string"}
#@markdown ---
# ACÁ CARGAMOS EL QUE VAMOS A CLASIFICAR
df_raw = pd.read_csv(Data)
df_raw = df_raw.rename(columns={'IDEA':'idea'})
df_raw = df_raw.dropna(subset=['idea'])
max_r_raw = 10000
df_raw = df_raw.iloc[:max_r_raw]
df_raw.head()

,country_name,location_name,service_point_name,service_type,satisfied,response_type,response_type_raw,satisfied_num,idea,response_lat,...,response_point,lat_lon,created_at_utc,uploaded_at_utc,time_zone,created_at_tz,uploaded_at_tz,unique_id,user_id,is_starred
0,Uganda_old,Bidi Bidi Zone 5,Ariwa 1 : Community Pulse,Community Pulse,False,Survey,NaN,0,Extend the group cash support to individuals o...,3.244788,...,"(31.38743,3.244788)","3.24478800,31.38743000",2023-05-22 06:42:19.000000 UTC,2023-05-22 11:38:52.000000 UTC,NaN,NaN,NaN,ccd42e00-f86b-11ed-ae16-e151ac19d791,59,True
1,Uganda_old,Bidi Bidi Zone 5,Ariwa 1 : Community Pulse,Community Pulse,False,Survey,NaN,0,Focus on giving cash for work projects so we c...,3.244788,...,"(31.38743,3.244788)","3.24478800,31.38743000",2023-05-22 06:57:41.000000 UTC,2023-05-22 11:38:52.000000 UTC,NaN,NaN,NaN,f2395240-f86d-11ed-ae16-e151ac19d791,59,False
2,Uganda_old,Bidi Bidi Zone 5,Ayivu : Community Pulse,Community Pulse,False,Survey,NaN,0,Cancel the program of school fees payment sinc...,3.253690,...,"(31.374929,3.25369)","3.25369000,31.37492900",2023-05-22 07:09:19.000000 UTC,2023-05-22 10:17:08.000000 UTC,NaN,NaN,NaN,91e389e0-f86f-11ed-b650-e951b6d2da8c,46,False
3,Uganda_old,Bidi Bidi Zone 5,Ayivu : Community Pulse,Community Pulse,False,Survey,NaN,0,Scale land support to each household to boost ...,3.253690,...,"(31.374929,3.25369)","3.25369000,31.37492900",2023-05-22 07:16:42.000000 UTC,2023-05-22 10:17:08.000000 UTC,NaN,NaN,NaN,9a3f2ad0-f870-11ed-b650-e951b6d2da8c,46,False
4,Uganda_old,Bidi Bidi Zone 5,Ayivu : Community Pulse,Community Pulse,True,Survey,NaN,1,Rise the cereals to 12kgs to normalise food in...,3.253690,...,"(31.374929,3.25369)","3.25369000,31.37492900",2023-05-22 07:20:39.000000 UTC,2023-05-22 10:17:08.000000 UTC,NaN,NaN,NaN,275f18d0-f871-11ed-b650-e951b6d2da8c,46,False


In [ ]:
#@title **6. Final model**
def find_relevant_subtopic(sim_df, TG_list, ST_list, TT_list, Prob_list, Probnum_list):
  threshold_1 = 0.80
  threshold_2 = 0.70
  threshold_3 = 0.60
  threshold_4 = 0.50
  mask = (sim_df['sim_score'] >= threshold_1)
  df_prov = sim_df[mask]
  if len(df_prov) == 0:
    mask = (sim_df['sim_score'] >= threshold_2) & (sim_df['sim_score'] < threshold_1)
    df_prov = sim_df[mask]
    if len(df_prov) == 0:
      mask = (sim_df['sim_score'] >= threshold_3) & (sim_df['sim_score'] < threshold_2)
      df_prov = sim_df[mask]
      if len(df_prov) == 0:
        mask = (sim_df['sim_score'] >= threshold_4) & (sim_df['sim_score'] < threshold_3)
        df_prov = sim_df[mask]
        if len(df_prov) == 0:
          mask = (sim_df['sim_score'] < threshold_4)
          df_prov = sim_df[mask]
          if len(df_prov) == 0:
            TG_list.append('New')
            ST_list.append('New')
            TT_list.append('New')
            Prob_list.append('Very Low')
            Probnum_list.append('New')
            return TG_list, ST_list, TT_list, Prob_list, Probnum_list
          else:
            TG_list.append('New')
            ST_list.append('New')
            TT_list.append('New')
            Prob_list.append('Very Low')
            Probnum_list.append('New')
            return TG_list, ST_list, TT_list, Prob_list, Probnum_list
        else:
          result = pd.pivot_table(df_prov, index = ["Sector", "Topic", "Subtopic"], aggfunc='mean').sort_values(by='sim_score', ascending=False)
          TG_list.append(result.iloc[[0]].reset_index(drop=False)['Sector'][0])
          ST_list.append(result.iloc[[0]].reset_index(drop=False)['Topic'][0])
          TT_list.append(result.iloc[[0]].reset_index(drop=False)['Subtopic'][0])
          Prob_list.append('Low')
          Probnum_list.append(result.iloc[[0]].reset_index(drop=False)['sim_score'][0])
          return TG_list, ST_list, TT_list, Prob_list, Probnum_list
      else:
        result = pd.pivot_table(df_prov, index = ["Sector", "Topic", "Subtopic"], aggfunc='mean').sort_values(by='sim_score', ascending=False)
        TG_list.append(result.iloc[[0]].reset_index(drop=False)['Sector'][0])
        ST_list.append(result.iloc[[0]].reset_index(drop=False)['Topic'][0])
        TT_list.append(result.iloc[[0]].reset_index(drop=False)['Subtopic'][0])
        Prob_list.append('Good')
        Probnum_list.append(result.iloc[[0]].reset_index(drop=False)['sim_score'][0])
        return TG_list, ST_list, TT_list, Prob_list, Probnum_list
    else:
      result = pd.pivot_table(df_prov, index = ["Sector", "Topic", "Subtopic"], aggfunc='mean').sort_values(by='sim_score', ascending=False)
      TG_list.append(result.iloc[[0]].reset_index(drop=False)['Sector'][0])
      ST_list.append(result.iloc[[0]].reset_index(drop=False)['Topic'][0])
      TT_list.append(result.iloc[[0]].reset_index(drop=False)['Subtopic'][0])
      Prob_list.append('High')
      Probnum_list.append(result.iloc[[0]].reset_index(drop=False)['sim_score'][0])
      return TG_list, ST_list, TT_list, Prob_list, Probnum_list
  else:
    result = pd.pivot_table(df_prov, index = ["Sector", "Topic", "Subtopic"], aggfunc='mean').sort_values(by='sim_score', ascending=False)
    TG_list.append(result.iloc[[0]].reset_index(drop=False)['Sector'][0])
    ST_list.append(result.iloc[[0]].reset_index(drop=False)['Topic'][0])
    TT_list.append(result.iloc[[0]].reset_index(drop=False)['Subtopic'][0])
    Prob_list.append('Very High')
    Probnum_list.append(result.iloc[[0]].reset_index(drop=False)['sim_score'][0])
    return TG_list, ST_list, TT_list, Prob_list, Probnum_list

In [ ]:
#@title **7. Classification process**
TG_list = []
ST_list = []
TT_list = []
Prob_list = []
Probnum_list = []
for i in range (len(df_raw)):
  question = df_raw['idea'].apply(lambda x: x.split('#')[0])[i]
  print(i)
  e_col = 'sentence_embedding_bert'
  sim_df_all = get_sim_df_for_string(predictions,e_col,question)
  sim_df_all.index = sim_df_all.idea
  sim_df_all = sim_df_all.sort_values('sim_score', ascending = False)
  sim_df = sim_df_all[sim_df_all['sim_score']>=0]
  TG_list, ST_list, TT_list, Prob_list, Probnum_list = find_relevant_subtopic(sim_df, TG_list, ST_list, TT_list, Prob_list, Probnum_list)

0


<ipython-input-8-01738096107f>:42: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  result = pd.pivot_table(df_prov, index = ["Sector", "Topic", "Subtopic"], aggfunc='mean').sort_values(by='sim_score', ascending=False)


1


<ipython-input-8-01738096107f>:50: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  result = pd.pivot_table(df_prov, index = ["Sector", "Topic", "Subtopic"], aggfunc='mean').sort_values(by='sim_score', ascending=False)


2


...

<ipython-input-8-01738096107f>:42: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  result = pd.pivot_table(df_prov, index = ["Sector", "Topic", "Subtopic"], aggfunc='mean').sort_values(by='sim_score', ascending=False)
<ipython-input-8-01738096107f>:42: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  result = pd.pivot_table(df_prov, index = ["Sector", "Topic", "Subtopic"], aggfunc='mean').sort_values(by='sim_score', ascending=False)


69
70


<ipython-input-8-01738096107f>:58: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  result = pd.pivot_table(df_prov, index = ["Sector", "Topic", "Subtopic"], aggfunc='mean').sort_values(by='sim_score', ascending=False)
<ipython-input-8-01738096107f>:50: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  result = pd.pivot_table(df_prov, index = ["Sector", "Topic", "Subtopic"], aggfunc='mean').sort_values(by='sim_score', ascending=False)


71
72


<ipython-input-8-01738096107f>:42: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  result = pd.pivot_table(df_prov, index = ["Sector", "Topic", "Subtopic"], aggfunc='mean').sort_values(by='sim_score', ascending=False)


In [ ]:
#@title **8. Creation of the new data base**
df_raw['Sector'] = TG_list
df_raw['Topic'] = ST_list
df_raw['Subtopic'] = TT_list
df_raw['Probability'] = Prob_list
df_raw['sim_score'] = Probnum_list

In [ ]:
#@title **9. Download results**
df_raw
df_raw.to_excel("ResultsCPEA.xlsx")